# RANDOM MATRIX VERSION

In [ ]:
import os
import shutil
import subprocess
import time
import math
import threading
import sys
import numpy as np
from scipy.ndimage import gaussian_filter
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor, as_completed
from numba import jit, prange, config
import io

# === Configuration ===
t_min, t_max = 0, 7
qubit = 4                        # System size = 2^qubit × 2^qubit
system_dim = 2 ** qubit          # Hamiltonian/state dimension
height_scale = 1
grid_spacing = 1
sigma = 1
fps = 60
max_workers = 8
num_frames = int(math.ceil(fps * (t_max - t_min)))
chunk_size = 10
# ======================

# --- Quantum Gates Setup ---
I = np.eye(2, dtype=np.complex128)
X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
Y = np.array([[0, -1j], [1j, 0]], dtype=np.complex128)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)

def tensor_product(operators):
    """Compute tensor product of multiple operators"""
    result = operators[0]
    for op in operators[1:]:
        result = np.kron(result, op)
    return result

# --- Hamiltonian Construction ---
def build_hamiltonian():
    # Generate random Pauli terms for qubit-sized system
    num_terms = 10
    H = np.zeros((system_dim, system_dim), dtype=np.complex128)
    
    for _ in range(num_terms):
        # Random Pauli operators for each qubit
        paulis = [np.random.choice([I, X, Y, Z]) for _ in range(qubit)]
        term = tensor_product(paulis)
        H += np.random.randn() * term
    
    return (H + H.conj().T) / 2  # Hermitian guarantee

# --- Quantum System Initialization ---
print(f"Initializing {system_dim}x{system_dim} quantum system...")
H = build_hamiltonian()
V_rand = tensor_product([np.random.randn(2,2) + 1j*np.random.randn(2,2) 
                        for _ in range(qubit)])
desired_avg = np.mean(np.abs(V_rand))

# --- Precompute Evolution ---
print("Precomputing time evolution...")
pre_start = time.time()

t_values = np.linspace(t_min, t_max, num_frames)
Z_all = np.empty((system_dim, system_dim, num_frames))
C_all = np.empty((system_dim, system_dim, num_frames))

for i, t in enumerate(t_values):
    U = expm(-1j * H * t)
    evolved = V_rand @ U
    
    # Normalized magnitude [0, 1]
    abs_data = np.abs(evolved)
    filtered = gaussian_filter(abs_data * height_scale, sigma=sigma)
    fmin, fmax = filtered.min(), filtered.max()
    Z_all[:, :, i] = (filtered - fmin) / (fmax - fmin) if fmax > fmin else 0
    
    # Phase processing
    phase = (np.angle(evolved) + np.pi) / (2 * np.pi)
    C_all[:, :, i] = gaussian_filter(phase, sigma=sigma) % 1

print(f"Precomputed {num_frames} frames in {time.time()-pre_start:.2f}s")

# --- Rendering Pipeline ---
def create_ffmpeg_pipe():
    return subprocess.Popen([
        'ffmpeg', '-y', '-loglevel', 'error',
        '-f', 'image2pipe',          # Input format
        '-framerate', str(fps),      # Input framerate
        '-c:v', 'png',               # Input codec for PNG frames
        '-i', '-',                   # Input from pipe
        '-c:v', 'libx264',           # Standard H.264 encoder
        '-crf', '18',                # High quality (0-51 scale, lower=better)
        '-preset', 'slower',         # Better compression efficiency
        '-tune', 'animation',        # Optimize for animated content
        '-profile:v', 'high',        # High profile for better quality
        '-bf', '2',                  # B-frames for compression
        '-pix_fmt', 'yuv420p',       # Widely compatible color format
        '-movflags', '+faststart',   # Web optimization
        'animation.mp4'
    ], stdin=subprocess.PIPE)

def render_chunk(frames, fig_template, pipe, progress_tracker):
    buffers = []
    
    for frame in frames:
        fig = go.Figure(fig_template)
        fig.add_trace(go.Surface(
            z=Z_all[:, :, frame],
            surfacecolor=C_all[:, :, frame],
            colorscale='HSV',
            cmin=0,
            cmax=1,
            showscale=False
        ))
        
        fig.update_layout(
            uirevision='constant',
            scene_camera=dict(projection=dict(type='orthographic')),
            title=f"Time Evolution: t = {t_values[frame]:.2f}"
        )
        
        buf = io.BytesIO()
        fig.write_image(buf, format='png', engine='kaleido')  # Fix here
        buffers.append(buf.getvalue())
        progress_tracker.update(1)
    
    pipe.write(b''.join(buffers))
    return len(frames)

# --- Main Workflow ---
if __name__ == "__main__":
    shutil.rmtree("frames", ignore_errors=True)
    
    fig_template = go.Figure().update_layout(
        scene=dict(
            xaxis_showspikes=False,
            yaxis_showspikes=False,
            zaxis_showspikes=False,
            xaxis_range=[0, s*grid_spacing],
            yaxis_range=[0, s*grid_spacing],
            zaxis_range=[0, np.max(Z_all)],
            aspectratio=dict(x=1, y=1, z=0.7),
            camera_projection=dict(type='orthographic')
        ),
        margin=dict(l=0, r=0, b=0, t=30),
        paper_bgcolor='white',  # Changed from transparent
        plot_bgcolor='white',   # Changed from transparent
        modebar_remove=['zoom3d', 'pan3d', 'resetCameraDefault3d']
    ).update_traces(
        lightposition=dict(x=0, y=0, z=0),
        lighting_roughness=1,
        lighting_specular=0,
        showlegend=False
    ).to_dict()
    
    chunks = [range(i, min(i+chunk_size, num_frames)) 
             for i in range(0, num_frames, chunk_size)]
    
    ffmpeg = create_ffmpeg_pipe()
    progress = ProgressTracker(num_frames)
    
    print(f"\nRendering {num_frames} frames ({len(chunks)} chunks)")
    print("Starting render...")
    render_start = time.time()
    
    try:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(render_chunk, chunk, fig_template, ffmpeg.stdin, progress): chunk 
                      for chunk in chunks}
            
            for future in as_completed(futures):
                future.result()
    finally:
        progress.update(0)
        ffmpeg.stdin.close()
        ffmpeg.wait()
    
    total_time = time.time() - render_start
    print(f"\n\nDone! Total render time: {total_time:.1f}s")
    print(f"Average speed: {num_frames/total_time:.1f} FPS")
    print("Output: animation.mp4")


Precomputing simulation data...
Data precomputed in 0.52s

Rendering 420 frames (42 chunks)
Starting render...
Rendering: |████████████████████████████████████████| 420/420 (100.0%) | 0.8 fps | Elapsed: 516.4s

Done! Total render time: 516.8s
Average speed: 0.8 FPS
Output: animation.mp4


# PSEUDO HAMOLTONIAN 

In [5]:
import os
import shutil
import subprocess
import time
import math
import threading
import sys
import numpy as np
from scipy.linalg import expm
from scipy.ndimage import gaussian_filter
import plotly.graph_objects as go
from concurrent.futures import ThreadPoolExecutor, as_completed

# === Configuration ===
t_min, t_max = 0, 7
n_qubits = 4                     # Number of qubits
system_dim = 2 ** n_qubits       # 16x16 system (256 elements)
height_scale = 1
grid_spacing = 1
sigma = 1
fps = 60
max_workers = 8
num_frames = int(math.ceil(fps * (t_max - t_min)))
chunk_size = 10
# ======================

# --- Quantum Gates Setup ---
I = np.eye(2, dtype=np.complex128)
X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
Y = np.array([[0, -1j], [1j, 0]], dtype=np.complex128)
Z = np.array([[1, 0], [0, -1]], dtype=np.complex128)

def tensor_product(operators):
    """Compute tensor product of multiple operators"""
    result = operators[0]
    for op in operators[1:]:
        result = np.kron(result, op)
    return result

def build_hamiltonian():
    pauli_gates = [I, X, Y, Z]  # List of available gates
    num_terms = 10
    H = np.zeros((system_dim, system_dim), dtype=np.complex128)
    
    for _ in range(num_terms):
        # Randomly select gate indices for each qubit
        gate_indices = np.random.randint(0, 4, size=n_qubits)
        paulis = [pauli_gates[i] for i in gate_indices]
        
        term = tensor_product(paulis)
        H += np.random.randn() * term
    
    return (H + H.conj().T) / 2  # Hermitian guarantee

# --- Quantum System Initialization ---
print("Initializing quantum system...")
H = build_hamiltonian()
V_rand = tensor_product([np.random.randn(2,2) + 1j*np.random.randn(2,2) 
                        for _ in range(n_qubits)])
desired_avg = np.mean(np.abs(V_rand))

# --- Precompute Evolution ---
print("Precomputing time evolution...")
pre_start = time.time()

t_values = np.linspace(t_min, t_max, num_frames)
Z_all = np.empty((system_dim, system_dim, num_frames))
C_all = np.empty((system_dim, system_dim, num_frames))

for i, t in enumerate(t_values):
    # Compute time evolution operator
    U = expm(-1j * H * t)
    evolved = V_rand @ U
    
    # Process magnitude with normalization
    abs_data = np.abs(evolved)
    filtered = gaussian_filter(abs_data * height_scale, sigma=sigma)
    
    # Normalize to [0, 1] per frame
    fmin, fmax = filtered.min(), filtered.max()
    if fmax > fmin:  # Only normalize if there's variation
        normalized = (filtered - fmin) / (fmax - fmin)
    else:
        normalized = np.zeros_like(filtered)
    Z_all[:, :, i] = normalized
    
    # Process phase (already normalized to [0, 1])
    phase = (np.angle(evolved) + np.pi) / (2 * np.pi)
    C_all[:, :, i] = gaussian_filter(phase, sigma=sigma) % 1

print(f"Precomputed in {time.time()-pre_start:.2f}s")

# --- Visualization Pipeline ---
class ProgressTracker(threading.Thread):
    def __init__(self, total):
        super().__init__()
        self.total = total
        self.count = 0
        self.start_time = time.time()
        
    def run(self):
        while self.count < self.total:
            elapsed = time.time() - self.start_time
            sys.stdout.write(
                f"\rRendering: {self.count}/{self.total} "
                f"({self.count/self.total:.1%}) | "
                f"Elapsed: {elapsed:.1f}s"
            )
            sys.stdout.flush()
            time.sleep(0.1)

def create_ffmpeg_pipe():
    return subprocess.Popen([
        'ffmpeg', '-y', '-loglevel', 'error',
        '-f', 'image2pipe',
        '-framerate', str(fps),
        '-c:v', 'png',
        '-i', '-',
        '-c:v', 'libx264',
        '-crf', '18',
        '-preset', 'slow',
        '-tune', 'animation',
        '-pix_fmt', 'yuv420p',
        '-movflags', '+faststart',
        'quantum_system.mp4'
    ], stdin=subprocess.PIPE)

def render_frame(frame, fig_template):
    fig = go.Figure(fig_template)
    fig.add_trace(go.Surface(
        z=Z_all[:, :, frame],
        surfacecolor=C_all[:, :, frame],
        colorscale='HSV',
        cmin=0,
        cmax=1,
        showscale=False
    ))
    fig.update_layout(
        scene_camera=dict(projection=dict(type='orthographic')),
        title=f"Qubit System Evolution: t = {t_values[frame]:.2f}"
    )
    buf = io.BytesIO()
    fig.write_image(buf, format='png', scale=2)
    return buf.getvalue()

# --- Main Execution ---
if __name__ == "__main__":
    shutil.rmtree("frames", ignore_errors=True)
    
    # Prepare template
    fig_template = go.Figure().update_layout(
        scene=dict(
            xaxis_showspikes=False,
            yaxis_showspikes=False,
            zaxis_showspikes=False,
            aspectratio=dict(x=1, y=1, z=0.7),
            camera_projection=dict(type='orthographic')
        ),
        margin=dict(l=0, r=0, b=0, t=30),
        paper_bgcolor='white',
        plot_bgcolor='white'
    ).to_dict()
    
    ffmpeg = create_ffmpeg_pipe()
    progress = ProgressTracker(num_frames)
    progress.start()
    
    try:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for frame in range(num_frames):
                futures.append(executor.submit(render_frame, frame, fig_template))
            
            for i, future in enumerate(futures):
                ffmpeg.stdin.write(future.result())
                progress.count = i + 1
                
    finally:
        progress.join()
        ffmpeg.stdin.close()
        ffmpeg.wait()
    
    print("\n\nRendering complete!")
    print("Output: quantum_system.mp4")

Initializing quantum system...
Precomputing time evolution...
Precomputed in 0.08s
Rendering: 419/420 (99.8%) | Elapsed: 788.3s

Rendering complete!
Output: quantum_system.mp4
